In [4]:
#Essentially a translator that takes your prompt and reformats for LangChain to better understand to process better
from langchain.adapters.openai import convert_openai_messages
from langchain_openai import ChatOpenAI
from tavily import TavilyClient
import os 

Tavily Search

In [5]:
client = TavilyClient(api_key=os.environ.get('TAVILY_API_KEY')) 
response = client.search(query= "Recommend me the top five fragrances")
#Returns sources sorted by relevant scores
print(response)

{'query': 'Recommend me the top five fragrances', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'These Are the Top 5 Fragrances Among NewBeauty Readers', 'url': 'https://www.newbeauty.com/top-5-favorite-fragrances-newbeauty-readers/', 'content': 'These Are the Top 5 Fragrances Among NewBeauty Readers - NewBeauty 2024 NewBeauty Readers Choice Awards These Are the Top 5 Fragrances Among NewBeauty Readers From a study of over 700 NewBeauty readers, we narrowed down respondents’ top five perfume brands, and the specific scents from those brands they love most*. Dior boasts several modern classics among their fragrances, but the two our readers cited most were floral favorites. Another bonafide celebrity favorite, Jo Malone London lands at number two among NewBeauty readers. NewBeauty readers’ go-to fragrance brand is Chanel, home of the perfume that has become synonymous with perfume itself, N°5 ($165). Combination Skin-Care Essentials NewBeauty Readers L

Processing & Displaying Results

In [6]:
for result in response['results']:
    print(result['title'])

These Are the Top 5 Fragrances Among NewBeauty Readers
20 Best Perfumes for Women in 2024 - Top Fragrances of All Time
The 27 Very Best Perfumes - New York Magazine
The 23 Best Perfumes and Fragrances For Women In 2024 - Women's Health
20 Best Perfumes for Women, Tested & Reviewed for 2024 - Harper's BAZAAR


More Advanced Search Query

In [7]:
response = client.search(
    query="Based on this user’s preferences, what are the top 5 bestselling cologne/perfumes? Please provide the results in a single string separated by a ‘/’ (e.g., fragrance1/fragrance2/fragrance3/fragrance4/fragrance5): 'Fragrance Type: Cologne Fragrance Format(s): Eau de Parfum (EDP), Eau de Toilette (EDT), Solid Perfumes Fragrance Portfolio(s): Woody, Oriental, Spicy'",
    #A more in depth search but provides more comprehensive results
    search_depth='advanced', 
    max_results=1, 
    include_images=True, 
)
for result in response['results']:
    print(result['content'])

Top Women Perfumes Perfumes Women Perfumes Eyes Eye Cream Eye Cream Hair Perfume Perfumes Perfumes French Fragrance offers you a detailed look at some of the best-selling and most iconic perfumes in the world. Their strong brand identities and enduring appeal highlight the diversity of popular scents in the fragrance world. Elie Saab Le Parfum For Women is a luxurious fragrance that blends confidence and charm with bold sophistication, which is specially catered . It brings together an essence of leather with floral, spicy, and woody notes, which can be worn by men and women both. Perfumes (82) Best Luxury Fragrances for New Year Gifting 2025 Top Branded Perfumes for Women in 2025 Top Branded Perfumes for Women French Fragrance Women Perfumes


Quick Direct Search Query (Concise and Factual) Probably best for this project

In [8]:
print(client.qna_search(query= "Based on this user’s preferences, what are the top 5 bestselling cologne/perfumes? Please provide the results in a single string separated by a ‘/’ (e.g., fragrance1/fragrance2/fragrance3/fragrance4/fragrance5): 'Fragrance Type: Cologne Fragrance Format(s): Eau de Parfum (EDP), Eau de Toilette (EDT), Solid Perfumes Fragrance Portfolio(s): Woody, Oriental, Spicy'"))

Based on the user's preferences for colognes in Woody, Oriental, and Spicy portfolios, the top 5 bestselling fragrances in 2025 are Chanel No. 19, Eros Intense, Dior Sauvage, Jo Malone London Green Tea, and Tom Ford Oud Wood.


Combining Tavily's search capabilities with LLMS using LangChain

In [9]:
#Essentially translates our query into a more formatted way for LangChain to understand
from langchain.adapters.openai import convert_openai_messages
#Allow us to interact with LLMS
from langchain_openai import ChatOpenAI


#Tavily Prompt (Search Query To Retrieve Data):
query = "Based on this user’s preferences, what are the top 5 bestselling cologne/perfumes? Please provide the results in a single string separated by a ‘/’ (e.g., fragrance1/fragrance2/fragrance3/fragrance4/fragrance5): 'Fragrance Type: Cologne Fragrance Format(s): Eau de Parfum (EDP), Eau de Toilette (EDT), Solid Perfumes Fragrance Portfolio(s): Woody, Oriental, Spicy'"
response = client.search(query, max_results= 10, search_depth="advanced")['results']
response

[{'title': 'Best-Selling Branded Perfumes in 2025 - frenchfragrance.com',
  'url': 'https://frenchfragrance.com/blog/best-selling-branded-perfumes-in-2025',
  'content': 'Top Women Perfumes Perfumes Women Perfumes Eyes Eye Cream Eye Cream Hair Perfume Perfumes Perfumes French Fragrance offers you a detailed look at some of the best-selling and most iconic perfumes in the world. Their strong brand identities and enduring appeal highlight the diversity of popular scents in the fragrance world. Elie Saab Le Parfum For Women is a luxurious fragrance that blends confidence and charm with bold sophistication, which is specially catered . It brings together an essence of leather with floral, spicy, and woody notes, which can be worn by men and women both. Perfumes (82) Best Luxury Fragrances for New Year Gifting 2025 Top Branded Perfumes for Women in 2025 Top Branded Perfumes for Women French Fragrance Women Perfumes',
  'score': 0.7680558,
  'raw_content': None},
 {'title': '10 Best Woody Pe

In [10]:
#LangChain Prompt (Search Query To Process Data): 
prompt = [
    #Define the AI's job description
    {
        'role':'system', 
        'content': f''' 
        You are an AI critical thinker research assistant. 
        Your purpose is to help recommend user's best selling fragrances.
        '''
    },
    #Give Specific instructions for the current task based on the input data
    {
        'role': 'user', 
        'content': f'''Information: """{response}"""
        Using the information above, answer the following query: """{query}""". 
        '''
    }
]

lc_messages = convert_openai_messages(prompt)
lc_messages

[SystemMessage(content=" \n        You are an AI critical thinker research assistant. \n        Your purpose is to help recommend user's best selling fragrances.\n        ", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Information: """[{\'title\': \'Best-Selling Branded Perfumes in 2025 - frenchfragrance.com\', \'url\': \'https://frenchfragrance.com/blog/best-selling-branded-perfumes-in-2025\', \'content\': \'Top Women Perfumes Perfumes Women Perfumes Eyes Eye Cream Eye Cream Hair Perfume Perfumes Perfumes French Fragrance offers you a detailed look at some of the best-selling and most iconic perfumes in the world. Their strong brand identities and enduring appeal highlight the diversity of popular scents in the fragrance world. Elie Saab Le Parfum For Women is a luxurious fragrance that blends confidence and charm with bold sophistication, which is specially catered . It brings together an essence of leather with floral, spicy, and woody notes, which can be worn

In [11]:
response = ChatOpenAI(model='gpt-4o-mini').invoke(lc_messages)
print(response.content)

Elie Saab Le Parfum For Women/Spice and Black Vanilla by Cremo/Erbario Toscano Spicy Vanilla Eau De Parfum/Boss Bottled/Nautica Voyage
